Machine Learning

1. Importamos librerias necesarias

In [2]:
import pandas as pd
import numpy as np 


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [22]:
from sklearn.neighbors import NearestNeighbors

In [4]:
#Abro los archivos a utilizar
total_plataformas = pd.read_csv('totalplataformas.csv', delimiter=',', encoding='utf-8')
total_score= pd.read_csv('score_movies.csv', delimiter=',', encoding='utf-8')

In [169]:
total_plataformas.head(2)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int,duration_type
0,ns1,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",90.0,min
1,ns2,s2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",2.0,season


In [170]:
total_score.head(2)

,id,score
0,as1,3.467131
1,as10,3.439571


In [7]:
total_score.rename({'movieId':'id'}, axis=1, inplace=True)#renombro id para unir df

In [9]:
df_completo = total_plataformas.merge(total_score,on='id', how='left')

In [11]:
df_completo.shape

(22998, 16)

In [29]:
#me quedo solo con las peliculas
df_peliculas= df_completo[df_completo.type=='movie']

In [30]:
df_peliculas.shape

(16481, 16)

In [68]:
df_peliculas.reset_index(inplace=True)#reseteo el indice para trabajar con el modelo

In [171]:
df_peliculas.to_csv('peliculas.csv', index=False) #guardo el archivo para las consultas en la Api

2. Realizo transformaciones necesarias
Sistema de recomendación en base a las características de la película (director, actores, género, puntuacion promedio).

In [35]:
# Filtro data a utilizar
columnas = ['director', 'cast', 'listed_in', 'score']
df_filt_peliculas = df_peliculas.loc[:,columnas]
df_filt_peliculas.head()

,director,cast,listed_in,score
0,kirsten johnson,NaN,documentaries,3.611111
6,"robert cullen, josé luis ucha","vanessa hudgens, kimiko glenn, james marsden, ...",children & family movies,3.482183
7,haile gerima,"kofi ghanaba, oyafunmike ogunlano, alexandra d...","dramas, independent movies, international movies",3.491736
9,theodore melfi,"melissa mccarthy, chris o'dowd, kevin kline, t...","comedies, dramas",3.571584
12,christian schwochow,"luna wedler, jannis niewöhner, milan peschel, ...","dramas, international movies",3.531915


In [38]:
df_filt_peliculas.isna().sum()

director        0
cast         2842
listed_in       0
score           0
dtype: int64

In [ ]:
df_filt_peliculas.director.fillna('sin registro', inplace=True)#lleno campos vacios

In [39]:
df_filt_peliculas.cast.fillna('sin registro', inplace=True)#lleno campos vacios

In [172]:
df_filt_peliculas.isna().sum()#verificamos

director     0
cast         0
listed_in    0
score        0
dtype: int64

In [210]:
df_filt_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16481 entries, 0 to 22997
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   director   16481 non-null  object 
 1   cast       16481 non-null  object 
 2   listed_in  16481 non-null  object 
 3   score      16481 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.1+ MB


In [175]:
df_filt_peliculas.to_csv('filtro_peliculas.csv', index=False)#guardo archivo para la Api

In [41]:
# Creo pipeline para variables numericas
pipeline_num = Pipeline([
    ('scaler', StandardScaler())
    ])

In [42]:
# Creo pipeline para variables categoricas
pipeline_categorica = Pipeline([
    ('encoder', OneHotEncoder(drop = 'first'))
    ])

In [43]:
# creo las transfromaciones
col_transf = ColumnTransformer([
    ('numeric', pipeline_num, df_filt_peliculas._get_numeric_data().columns.tolist()),
    ('categoric', pipeline_categorica, df_filt_peliculas.select_dtypes('object').columns.tolist()) 
    ])

In [44]:
#entrenamos
col_transf_fit = col_transf.fit(df_filt_peliculas)
df_filt_transf = col_transf_fit.transform(df_filt_peliculas)
df_filt_transf

<16481x23968 sparse matrix of type '<class 'numpy.float64'>'
	with 65672 stored elements in Compressed Sparse Row format>

In [152]:
#knn vecinos,  distancia coseno, entrenamos.
n_neighbors=6
nneighbors = NearestNeighbors(n_neighbors = n_neighbors, metric = 'cosine').fit(df_filt_transf)

In [127]:
#pruevo buscar el indice
titulo='sankofa'
indice=df_peliculas[df_peliculas.title==titulo].index.values[0]
indice

2

In [211]:
df_peliculas.head(3) #verifico

,index,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int,duration_type,score
0,0,ns1,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",90.0,min,3.611111
1,6,ns7,s7,movie,my little pony: a new generation,"robert cullen, josé luis ucha","vanessa hudgens, kimiko glenn, james marsden, ...",NaN,2021-09-24,2021,pg,91 min,children & family movies,equestria's divided. but a bright-eyed hero be...,91.0,min,3.482183
2,7,ns8,s8,movie,sankofa,haile gerima,"kofi ghanaba, oyafunmike ogunlano, alexandra d...","united states, ghana, burkina faso, united kin...",2021-09-24,1993,tv-ma,125 min,"dramas, independent movies, international movies","on a photo shoot in ghana, an american model s...",125.0,min,3.491736


In [168]:
#pruevo buscar por pelicula
titulo='the starling'
indice=df_peliculas[df_peliculas.title==titulo].index.values[0]#busco el indice
dif, ind = nneighbors.kneighbors(df_filt_transf[indice])#vecinos segun el indice dado


df_peliculas.loc[ind[0][0], :]#localizo el indice en el df gral

recomendado=df_peliculas.loc[ind[0][1:], :].sort_values('score', ascending=False)#localizo y acomodo segun el score
# lista = recomendado.title.to_list()
# lista
recomendado

,index,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int,duration_type,score
836,1262,ns1263,s1263,movie,ginny & georgia - the afterparty,NaN,"david spade, london hughes, fortune feimster, ...",NaN,2021-02-26,2021,tv-ma,35 min,"comedies, dramas",cast members of the hit netflix show join the ...,35.0,min,3.653608
3899,6006,ns6007,s6007,movie,28 days,betty thomas,"sandra bullock, viggo mortensen, dominic west,...",united states,2020-09-30,2000,pg-13,104 min,"comedies, dramas",after her drunken antics result in property da...,104.0,min,3.615312
3391,5282,ns5283,s5283,movie,carrie pilby,susan johnson,"bel powley, nathan lane, vanessa bayer, gabrie...",united states,2017-09-05,2016,tv-ma,98 min,"comedies, dramas",a socially awkward 19-year-old genius makes bi...,98.0,min,3.615152
5759,8352,ns8353,s8353,movie,the hundred-foot journey,lasse hallström,"helen mirren, om puri, manish dayal, charlotte...","united states, india, united arab emirates",2019-01-31,2014,pg,123 min,"comedies, dramas",a chef and his family leave india to open an e...,123.0,min,3.603272
3299,5129,ns5130,s5130,movie,el camino christmas,david e. talbert,"jessica alba, tim allen, dax shepard, vincent ...",united states,2017-12-08,2017,tv-ma,89 min,"comedies, dramas",stuck in a liquor store during an alleged robb...,89.0,min,3.586100


In [206]:
def get_recommendation(titulo: str):
  
    if df_peliculas.title[df_peliculas.title==titulo].empty:
        return 'pelicula no encontrada'  
    else:  
        indice=df_peliculas[df_peliculas.title==titulo].index.values[0]
        dif, ind = nneighbors.kneighbors(df_filt_transf[indice])

        df_peliculas.loc[ind[0][0], :]

        recomendado=df_peliculas.loc[ind[0][1:], :].sort_values('score', ascending=False)
        lista = recomendado.title.to_list()

        return {'recomendacion': lista}



In [205]:
df_peliculas.title[df_peliculas.title=='fdg']

Series([], Name: title, dtype: object)

In [208]:
get_recommendation('the starling') #verifico funcion

{'recomendacion': ['ginny & georgia - the afterparty',
  '28 days',
  'carrie pilby',
  'the hundred-foot journey',
  'el camino christmas']}